<h1 align="center"><font size="7">Deep Learning Project</font></h1>

# Introduction

The objective is to classify cat vs non-cat images

# Importing Packages

In [6]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
import copy
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
np.random.seed(1)

import os
from PIL import Image 

In [7]:
image_size = (64,64) 
batch_size = 30

In [8]:
train_data_dir = 'E:/Project1/data/train'
true_data_dir = 'E:/Project1/data/validation'

In [9]:
def load_and_preprocess_data(train_data_dir):
    
    flattened_images = []
    target_size = (64, 64)
    for filename in os.listdir(train_data_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(train_data_dir, filename)
            image = Image.open(image_path)
            image_resized = image.resize(target_size)
            image_array = np.array(image_resized)
            flattened_image = image_array.reshape(-1)
            flattened_images.append(flattened_image)
    flattened_images_array = np.array(flattened_images)
    flattened_images_array = flattened_images_array.T
    print("Original Images Shape:", (len(flattened_images),) + image_array.shape)
    print("Flattened Images Shape:", flattened_images_array.shape)
    train_x = flattened_images_array/255
    return train_x
check = load_and_preprocess_data(train_data_dir)

Original Images Shape: (200, 64, 64, 3)
Flattened Images Shape: (12288, 200)


In [10]:
def initialize_parameters(layer_dims):
    np.random.seed(42)
    parameters = {}
    L= len(layer_dims)
    for l in range(1,L):
        parameters['W' + str(l)] =(np.random.randn(layer_dims[l], layer_dims[l-1]))*np.sqrt(2/layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        assert(parameters['W' +str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' +str(l)].shape == (layer_dims[l], 1))
    return parameters

In [19]:
layer_dims = [12288, 520, 415, 337, 221, 103, 88, 56, 21, 1]
parameters = initialize_parameters(layer_dims)

for key, value in parameters.items():
#     print(key,value.shape)

# print("Test_Case : \n")
# print("W1 =" +str(parameters["W1"]))
# print("W2 =" +str(parameters["W2"]))
# print("b1 =" +str(parameters["b1"]))
# print("b2 =" +str(parameters["b2"]))


SyntaxError: incomplete input (1543753024.py, line 11)

In [12]:

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache
def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = np.dot(W,A_prev)+b, (A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = np.dot(W,A_prev)+b, (A_prev, W, b)
        A, activation_cache = relu(Z)
                
    cache = (linear_cache, activation_cache) 
    return A, cache

def L_layer_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters)//2
    for l in range (1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [13]:
train_data_dir = 'E:/Project1/data/train'
X = load_and_preprocess_data(train_data_dir)
layer_dims = [12288, 520, 415, 337, 221, 103, 88, 56, 21, 1]
parameters = initialize_parameters(layer_dims)

AL, caches = L_layer_forward(X, parameters)
#print("AL =" + str(AL))

Original Images Shape: (200, 64, 64, 3)
Flattened Images Shape: (12288, 200)


In [14]:
def create_true_label_vector(true_data_dir):
    label_map = {'cats': 1, 'dogs': 0} 
    Y = []
    for subfolder in os.listdir(true_data_dir):
        subfolder_path = os.path.join(true_data_dir, subfolder)
        if os.path.isdir(subfolder_path):
            label = label_map[subfolder.lower()]  
            for filename in os.listdir(subfolder_path):
                Y.append(label)

    
    Y = np.array(Y)

    return Y
Y = create_true_label_vector(true_data_dir)
#print("True label vector shape:", Y.shape)


In [15]:
def compute_cost(AL, Y):
    Y = Y.reshape(1, -1)
    m = Y.shape[1]
    cost = -1/m*(np.sum(np.multiply(Y, np.log(AL))+np.multiply((1-Y), np.log(1-AL))))
    cost = np.squeeze(cost)
    return cost
cost = compute_cost(AL, Y)
print("cost :" + str(cost))

cost :0.7424739073515859


In [16]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy= True)
    dZ[Z<0]=0
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA*s*(1-s)
    return dZ

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dA_prev = np.dot(W.T, dZ)
    dW = 1/m*np.dot(dZ, A_prev.T)
    db = 1/m*np.sum(dZ, axis =1, keepdims = True)
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db


def L_layer_backward(AL, Y, caches):
    grads={}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = -(np.divide(Y,AL)-np.divide((1-Y), (1-AL)))
    
    current_cache = caches[L-1]
    grads["dA"+str(L-1)], grads["dW"+str(L)], grads["db"+str(L)] = linear_activation_backward(dAL, current_cache, activation= "sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" +str(l+1)], current_cache, activation= "relu")
        grads["dA"+str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db"+str(l+1)] = db_temp
        
    return grads
    



grads = L_layer_backward(AL, Y, caches)

# print("dA0 = " + str(grads['dA0']))
# print("dA1 = " + str(grads['dA1']))
# print("dW1 = " + str(grads['dW1']))
# print("dW2 = " + str(grads['dW2']))
# print("db1 = " + str(grads['db1']))
# print("db2 = " + str(grads['db2']))
    

In [17]:
def update_parameters(parameters, grads, learning_rate):
   # params = copy.deepcopy(parameters)
    L = len(parameters)//2
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l+1)]
    return parameters
parameters = update_parameters(parameters, grads, 0.1)

# print ("W1 = "+ str(parameters["W1"]))
# print ("b1 = "+ str(parameters["b1"]))
# print ("W2 = "+ str(parameters["W2"]))
# print ("b2 = "+ str(parameters["b2"]))

In [18]:
def L_layer_model(X, Y, layer_dims, learning_rate = 0.01, num_iterations = 3000, print_cost = False):
    np.random.seed(3)
    costs = []
    parameters = initialize_parameters(layer_dims)
    for i in range(0, num_iterations):
        AL, caches = L_layer_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_layer_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    return parameters, costs

parameters, costs = L_layer_model(X, Y, layer_dims=[12288, 520, 415, 337, 221, 103, 88, 56, 21, 1], num_iterations = 2500, print_cost = True)

Cost after iteration 0: 0.7424739073515859
Cost after iteration 100: 0.5281046705897657
Cost after iteration 200: 0.3196576472070168
Cost after iteration 300: 0.019269754435368176
Cost after iteration 400: 0.005631689071321864
Cost after iteration 500: 0.0029561792324722686
Cost after iteration 600: 0.0019137595113664888
Cost after iteration 700: 0.0013807705624944316
Cost after iteration 800: 0.001061128671385537
Cost after iteration 900: 0.0008520064160434341
Cost after iteration 1000: 0.0007057179969220745
Cost after iteration 1100: 0.0005985065294594843
Cost after iteration 1200: 0.0005168561283113715
Cost after iteration 1300: 0.0004532183783470804
Cost after iteration 1400: 0.0004023329820501538
Cost after iteration 1500: 0.00036074834847946716
Cost after iteration 1600: 0.00032633658352151376
Cost after iteration 1700: 0.00029747617574257454
Cost after iteration 1800: 0.0002727732047828843
Cost after iteration 1900: 0.0002515363455905111
Cost after iteration 2000: 0.000233115992